In [3]:
import cplex
from cplex.exceptions import CplexError
import sys


In [9]:
# data common to all populateby functions
my_obj      = [1.0, 2.0, 3.0]
my_ub       = [40.0, cplex.infinity, cplex.infinity]
my_colnames = ["x1", "x2", "x3"]
my_rhs      = [20.0, 30.0]
my_rownames = ["c1", "c2"]
my_sense    = "LL"


def populatebyrow(prob):
    prob.objective.set_sense(prob.objective.sense.maximize)

    # since lower bounds are all 0.0 (the default), lb is omitted here
    prob.variables.add(obj = my_obj, ub = my_ub, names = my_colnames)

    # can query variables like the following bounds and names:

    # lbs is a list of all the lower bounds
    lbs = prob.variables.get_lower_bounds()

    # ub1 is just the first lower bound
    ub1 = prob.variables.get_upper_bounds(0)

    # names is ["x1", "x3"]
    names = prob.variables.get_names([0, 2])

    rows = [[[0,"x2","x3"],[-1.0, 1.0,1.0]],
            [["x1",1,2],[ 1.0,-3.0,1.0]]]

    prob.linear_constraints.add(lin_expr = rows, senses = my_sense,
                                rhs = my_rhs, names = my_rownames)

    # because there are two arguments, they are taken to specify a range
    # thus, cols is the entire constraint matrix as a list of column vectors
    cols = prob.variables.get_cols("x1", "x3")


def populatebycolumn(prob):
    prob.objective.set_sense(prob.objective.sense.maximize)

    prob.linear_constraints.add(rhs = my_rhs, senses = my_sense,
                                names = my_rownames)

    c = [[[0,1],[-1.0, 1.0]],
         [["c1",1],[ 1.0,-3.0]],
         [[0,"c2"],[ 1.0, 1.0]]]

    prob.variables.add(obj = my_obj, ub = my_ub, names = my_colnames,
                       columns = c)

def populatebynonzero(prob):
    prob.objective.set_sense(prob.objective.sense.maximize)

    prob.linear_constraints.add(rhs = my_rhs, senses = my_sense,
                                names = my_rownames)
    prob.variables.add(obj = my_obj, ub = my_ub, names = my_colnames)

    rows = [0,0,0,1,1,1]
    cols = [0,1,2,0,1,2]
    vals = [-1.0,1.0,1.0,1.0,-3.0,1.0]

    prob.linear_constraints.set_coefficients(zip(rows, cols, vals))
    # can also change one coefficient at a time

    # prob.linear_constraints.set_coefficients(1,1,-3.0)
    # or pass in a list of triples
    # prob.linear_constraints.set_coefficients([(0,1,1.0), (1,1,-3.0)])


def lpex1(pop_method):
    try:
        my_prob = cplex.Cplex()

        if pop_method == "r":
            handle = populatebyrow(my_prob)
        if pop_method == "c":
            handle = populatebycolumn(my_prob)
        if pop_method == "n":
            handle = populatebynonzero(my_prob)

        my_prob.solve()
    except exc:
        print(exc)
        return

    numrows = my_prob.linear_constraints.get_num()
    numcols = my_prob.variables.get_num()

    print
    # solution.get_status() returns an integer code
    print("Solution status = " , my_prob.solution.get_status(), ":",)
    # the following line prints the corresponding string
    print(my_prob.solution.status[my_prob.solution.get_status()])
    print("Solution value  = ", my_prob.solution.get_objective_value())
    slack = my_prob.solution.get_linear_slacks()
    pi    = my_prob.solution.get_dual_values()
    x     = my_prob.solution.get_values()
    dj    = my_prob.solution.get_reduced_costs()
    for i in range(numrows):
        print("Row %d:  Slack = %10f  Pi = %10f" % (i, slack[i], pi[i]))
    for j in range(numcols):
        print("Column %d:  Value = %10f Reduced cost = %10f" % (j, x[j], dj[j]))

    my_prob.write("lpex1.lp")



In [12]:
prompt = """Enter the letter indicating how the problem data should be populated:
r : populate by rows
c : populate by columns
n : populate by nonzeros\n ? > """
r = 'r'
c = 'c'
n = 'n'

lpex1(r)

Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
No LP presolve or aggregator reductions.
Presolve time = 0.00 sec. (0.00 ticks)

Iteration log . . .
Iteration:     1   Dual infeasibility =             0.000000
Iteration:     2   Dual objective     =           202.500000
Solution status =  1 :
optimal
Solution value  =  202.5
Row 0:  Slack =   0.000000  Pi =   2.750000
Row 1:  Slack =   0.000000  Pi =   0.250000
Column 0:  Value =  40.000000 Reduced cost =   3.500000
Column 1:  Value =  17.500000 Reduced cost =   0.000000
Column 2:  Value =  42.500000 Reduced cost =   0.000000


In [15]:
cplex.Cplex().get_version()

'12.10.0.0'